<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 Análisis de sentimentos y textos</p>
<p style="margin: 0; text-align:right;">Máster Universitario en Ciencia de Datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


# PAC 1: Procesamiento y análisis de información textual

En esta práctica revisaremos y aplicaremos los conocimientos aprendidos en los módulos del 1 al 2. Concretamente trataremos 3 temas.

<ul>
<li>1. Obtención de datos a partir de información textual
<li>2. Detección de tópicos
<li>3. Clasificación de textos
</ul>

El propósito de la práctica es descubrir rasgos característicos de las opiniones sobre restaurantes con las herramientas explicadas en los módulos del 1 al 2. Además, veremos si es posible clasificar automáticamente una opinión como positiva o negativa con métodos de machine learning. Utilizaremos el dataset <i>restaurants_reviews.csv</i>, extraído de una plataforma de expresión de opiniones. Este dataset contiene opiniones sobre restaurantes en inglés. El dataset se organiza en 10 columnas:

<b>business_id</b>: identificador del restaurant<br>
<b>date</b>: fecha de publicación de la opinión<br>
<b>review_id</b>: identificador de la opinión<br>
<b>stars</b>: calificación o valoración del restaurant en estrellas<br>
<b>text</b>: texto de la opinión<br>
<b>type</b>: tipo de texto<br>
<b>user_id</b>: identificador de usuario<br>
<b>cool, useful </b> y <b>funny</b>: Número de valoraciones que han realizado los usuarios de la plataforma para estos tres criterios.

In [24]:
# Desactivar els avisos
import warnings
warnings.filterwarnings('ignore')
import numpy as np

# Preparación del dataset

In [12]:
import pandas as pd
# Obrir el fitxer de comentaris:
df = pd.read_csv('restaurants_reviews.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simply put, beautiful. Full wi...",review,sqYN3lNgvPbPCTRsMFu27g,4,3,1
4,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4


Para realizar la práctica, sólo necesitaremos los textos y las valoraciones en estrellas. Por tanto, eliminamos las columnas innecesarias y nos quedaremos solo con las columnas <b>stars</b> y <b>text</b>.

In [13]:
df.drop(['business_id', 'review_id', 'user_id', 'date', 'type', 'funny', 'cool', 'useful'], axis=1, inplace=True)
df.head()

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,4,"Quiessence is, simply put, beautiful. Full wi..."
4,5,Drop what you're doing and drive here. After I...


Trabajaremos con las opiniones que tienen las calificaciones más altas y las más bajas. Las opiniones cuya calificación sea mayor que 3 serán etiquetadas con '1', mientras que etiquetaremos con '0' las opiniones que tengan una calificación menor a 3. Las etiquetas se asignan a la columna <b>sentiment</b>.

In [14]:
# Se eliminan las calificaciones que sean igual a tres
df = df[(df['stars'] > 3) | (df['stars'] < 3)] # Igual a df[(df['stars'] != 3)]

df['sentiment'] = df['stars'].apply(lambda x : 1 if x > 3 else 0)
df.head()

,stars,text,sentiment
0,5,My wife took me here on my birthday for breakf...,1
1,5,I have no idea why some people give bad review...,1
2,4,love the gyro plate. Rice is so good and I als...,1
3,4,"Quiessence is, simply put, beautiful. Full wi...",1
4,5,Drop what you're doing and drive here. After I...,1


# Preprocesamiento

Antes de trabajar con los textos de las opiniones, hay que limpiarlos de caracteres como los saltos de línea (e.g: *Should be called house of deliciousness!\r\n\r*)

In [15]:
class CarriageReturnReplacer(object):
    """ Replaces \r\n expressions in a text.
    >>> replacer = CarriageReturnReplacer()
    >>> replacer.replace("\r\n\r\nAnyway, I can\'t wait to go back!")
    'Anyway, I can\'t wait to go back!'
    """
    
    def replace(self, text):
        s = text
        # También se puede usar re.sub(r"(\r|\n)", "", s)
        s = s.replace('\r\n', ' ')
        s = s.replace('\n\n', ' ') 
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ') 
        return s

newline_replacer = CarriageReturnReplacer()

In [16]:
df['text'] = df['text'].apply(newline_replacer.replace)
df.head()

,stars,text,sentiment
0,5,My wife took me here on my birthday for breakf...,1
1,5,I have no idea why some people give bad review...,1
2,4,love the gyro plate. Rice is so good and I als...,1
3,4,"Quiessence is, simply put, beautiful. Full wi...",1
4,5,Drop what you're doing and drive here. After I...,1


También es necesario eliminar los doble espacios:

In [17]:
import re

class ExtraSpacesReplacer(object):
    """ Replaces extra spaces in a text.
    >>> replacer = ExtraSpacesReplacer()
    >>> replacer.replace("and it was excellent.  The weather was perfect")
    'and it was excellent. The weather was perfect'
    """
    
    def replace(self, text):
        s = text
        s = re.sub('\s\s+', ' ', s)
        return s

spaces_replacer = ExtraSpacesReplacer()

In [18]:
df['text'] = df['text'].apply(spaces_replacer.replace)
df.head()

,stars,text,sentiment
0,5,My wife took me here on my birthday for breakf...,1
1,5,I have no idea why some people give bad review...,1
2,4,love the gyro plate. Rice is so good and I als...,1
3,4,"Quiessence is, simply put, beautiful. Full win...",1
4,5,Drop what you're doing and drive here. After I...,1


# 1. Obtención de datos a partir de información textual (4 puntos)

## 1.1 Encontrar colocaciones (2 puntos)

Recordemos que las colocaciones son términos multipalabra, es decir, secuencias de palabras que, en conjunto, tienen un significado que difiere significativamente del significado de cada palabra individual (e.g. New York tiene un significado distinto del que se puede derivar de New y de York).

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong>  Calcula los mejores bigramas y trigramas de las opiniones. (1 punto)
</div>

In [41]:
# Para este apartado es necesario cargar las siguientes librerías:
import nltk
#nltk.download('all')
from nltk import pos_tag, word_tokenize
from nltk.collocations import *
import re

In [11]:
# Importar la lista de stopwords en inglés de la libreria NLTK.
stopwords = nltk.corpus.stopwords.words('english')
# Añadir stopwords
stopwords = stopwords + ['unknown', 've', 'hadn', 'll', 'didn', 'isn', 'doesn', 'hasn' ]

A partir del comando help(nltk.collocations.BigramAssocMeasures) explora la clase BigramAssocMeasures del módulo nltk.metrics.association y revisa las definiciones de las métricas de Likelihood Ratio (likelihood_ratio) y de Pointwise Mutual Information (pmi) se explica en el capítulo 5 del libro Foundations of Statistical Natural Language Processing (Manning & Schutze).

In [12]:
help(nltk.collocations.BigramAssocMeasures)

Help on class BigramAssocMeasures in module nltk.metrics.association:

class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |  - n_ii counts ``(w1, w2)``, i.e. the bigram being scored
 |  - n_ix counts ``(w1, *)``
 |  - n_xi counts ``(*, w2)``
 |  - n_xx counts ``(*, *)``, i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |  

<i>Primer paso</i>: Obtener los tokens del texto de las opiniones. Etiqueta estos tokens por su PoS. (0.5 puntos)

In [13]:
# Creamos texto en minúscula que recoja todas las opiniones
opinions = " ".join(df['text']).lower()
opinions[:500]

"my wife took me here on my birthday for breakfast and it was excellent. the weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. our waitress was excellent and our food arrived quickly on the semi-busy saturday morning. it looked like the place fills up pretty quickly so the earlier you get here the better. do yourself a favor and get their bloody mary. it was phenomenal and simply the best i've ever had. i'm pretty sure they only use ingredients from th"

In [14]:
#############################################
# SOLUCIÓN                                   #
#############################################
# Dividir las opiniones en tokens
opinions_tokens = [word for word in word_tokenize(opinions)]

# PoS tagging de los tokens
opinions_tagged_tokens = nltk.pos_tag(opinions_tokens)

In [68]:
opinions_tagged_tokens[0:10]

[('my', 'PRP$'),
 ('wife', 'NN'),
 ('took', 'VBD'),
 ('me', 'PRP'),
 ('here', 'RB'),
 ('on', 'IN'),
 ('my', 'PRP$'),
 ('birthday', 'NN'),
 ('for', 'IN'),
 ('breakfast', 'NN')]

<i>Segundo paso</i>: Calcular los 1000 mejores bigramas y los 1000 mejores trigramas a partir de los tokens etiquetados (e.g. [(Basic, JJ), ...]) del texto. Utiliza la métrica PMI o Likehood Ratio. Tienes que indicar por qué has escogido una y no otra. (0.5 puntos)

<b>Atención</b>: De los 1000 bigramas y trigramas, elige a los que no comienzan ni terminan con una stopword.

Recordemos la clasificación de etiquetas PoS.

<b>Etiquetas PoS</b>

<ul>
<li>DT: Determinante</li>
<li>JJ: Adjetivo</li>
<li>NN: Nombre en singular</li>
<li>NNS: Nombre en plural</li>
<li>VBD: Verbo en pasado</li>
<li>VBG: Verbo en gerundio</li>
<li>MD: Verbo modal</li>
<li>IN: Preposición o conjunción subordinada</li>
<li>PRP: Pronombre</li>
<li>RB: Adverbio</li>
<li>RP: Partícula</li>    
<li>CC: Conjunción coordinada</li>
<li>CD: Numeral</li>
</ul>

https://stats.stackexchange.com/questions/179010/difference-between-pointwise-mutual-information-and-log-likelihood-ratio

In [16]:
#############################################
# SOLUCIÓN                                  #
#############################################
# Cargamos las métricas para el cálculo de bigramas y trigramas
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# Cargamos ngrams para enseñar ejeplos de ngrmas
from nltk.util import ngrams

In [17]:
# Ejemplo de bigramas
opinion_bigrams = ngrams(opinions_tokens, 2)
print("BIGRAMAS")
print(str(list(opinion_bigrams)[0:15]))

# Ejemplo de trigramas
opinion_trigrams = ngrams(opinions_tokens, 3)
print()
print("TRIGRAMAS")
print(str(list(opinion_trigrams)[0:15]))

BIGRAMAS
[('my', 'wife'), ('wife', 'took'), ('took', 'me'), ('me', 'here'), ('here', 'on'), ('on', 'my'), ('my', 'birthday'), ('birthday', 'for'), ('for', 'breakfast'), ('breakfast', 'and'), ('and', 'it'), ('it', 'was'), ('was', 'excellent'), ('excellent', '.'), ('.', 'the')]

TRIGRAMAS
[('my', 'wife', 'took'), ('wife', 'took', 'me'), ('took', 'me', 'here'), ('me', 'here', 'on'), ('here', 'on', 'my'), ('on', 'my', 'birthday'), ('my', 'birthday', 'for'), ('birthday', 'for', 'breakfast'), ('for', 'breakfast', 'and'), ('breakfast', 'and', 'it'), ('and', 'it', 'was'), ('it', 'was', 'excellent'), ('was', 'excellent', '.'), ('excellent', '.', 'the'), ('.', 'the', 'weather')]


In [18]:
# Cálculo del 1000 mejores bigramas
bigram_candidates = BigramCollocationFinder.from_words(opinions_tokens)
best_1000_bigram_candidates = bigram_candidates.nbest(bigram_measures.pmi, 1000)
print("1000 mejores bigramas")
print(list(best_1000_bigram_candidates)[1:30])
print()

# Cálculo del 1000 mejores trigramas
trigram_candidates = TrigramCollocationFinder.from_words(opinions_tokens)
best_1000_trigram_candidates = trigram_candidates.nbest(trigram_measures.pmi, 1000)
print("1000 mejores trigramas")
print(list(best_1000_trigram_candidates)[1:30])

1000 mejores bigramas
[("'eggless", 'eggrolss'), ("'how", "'bout"), ("'real", "food'/a"), ("'visual", 'fluffers'), ('-50', 'gauges'), ('-elizabeth', 'rozin'), ('-or-', 'tinga'), ('.............', 'yeppie'), ('/glenlivit', '/glenmorangie'), ('1/16', 'teaspoon'), ('1202', 'mohave'), ('1980s', 'pseudo-asian'), ('1min', '45sec'), ('2-4-1.', 'woot'), ('4/', 'beach-head'), ('49er', 'flapjacks'), ('4th-tier', 'mid-90'), ('50¢', 'addtl'), ('65+', 'eyefucking'), ('=shiner', 'bock'), ('aa', 'meeting-'), ('abe', 'frohman'), ('administrative', 'assistant'), ('advent', 'epicurean'), ('afterglow', 'punctuated'), ('alain', 'keller'), ('alessandro', 'marchesan'), ('altering', 'drugs'), ('ambience=four', 'estrellas')]

1000 mejores trigramas
[('4/', 'beach-head', '2000'), ('bahay', 'kubo', 'natin'), ('bar/boxing', 'ring/vintage', 'store/food'), ('canh', 'chua', 'tom.i'), ('cheese/jewish', 'sliders/patty', 'melt/kosher'), ('dead-eyed', 'gamblers', 'shuffling'), ('donec', 'obviam', 'redimus'), ('egg-fooo

<div style="color:blue">Para filtrar las stopwords de los candidados, emplearemos las funciones definidas en la <b>PLA1</b> de la asignatura (Módulo 1-Cómo interpretar y analizar automáticamente la información textual, Joaquim More).</div>

In [19]:
# Función para filtrar los tokens que sean stopwords
def stopword_candidate(candidate):
    test = True
    if candidate[0] in stopwords or candidate[-1] in stopwords:
        test = False
    return test

# Función para filtrar los tokens
def filter_candidates(candidates, stopwords):
    #Si hemos cargado una lista de stopwords
    if len(stopwords) > 0:
         #Creamos una lista de candidatos que pasan el test de stopwords
        filtered_candidates = [candidate for candidate in candidates if stopword_candidate(candidate) == True]
    else:
        filtered_candidates = candidates
    return filtered_candidates

In [20]:
# Filtrar bigramas que comiencen por una stopword
best_1000_bigram_candidates_filtered = filter_candidates(best_1000_bigram_candidates, stopwords)
print("1000 mejores bigramas filtrados")
print(list(best_1000_bigram_candidates_filtered)[1:30])
print()

# Filtrar trigramas que comiencen por una stopword
best_1000_trigram_candidates_filtered = filter_candidates(best_1000_trigram_candidates, stopwords)
print("1000 mejores trigramas filtrados")
print(list(best_1000_trigram_candidates_filtered)[1:30])

1000 mejores bigramas filtrados
[("'eggless", 'eggrolss'), ("'how", "'bout"), ("'real", "food'/a"), ("'visual", 'fluffers'), ('-50', 'gauges'), ('-elizabeth', 'rozin'), ('-or-', 'tinga'), ('.............', 'yeppie'), ('/glenlivit', '/glenmorangie'), ('1/16', 'teaspoon'), ('1202', 'mohave'), ('1980s', 'pseudo-asian'), ('1min', '45sec'), ('2-4-1.', 'woot'), ('4/', 'beach-head'), ('49er', 'flapjacks'), ('4th-tier', 'mid-90'), ('50¢', 'addtl'), ('65+', 'eyefucking'), ('=shiner', 'bock'), ('aa', 'meeting-'), ('abe', 'frohman'), ('administrative', 'assistant'), ('advent', 'epicurean'), ('afterglow', 'punctuated'), ('alain', 'keller'), ('alessandro', 'marchesan'), ('altering', 'drugs'), ('ambience=four', 'estrellas')]

1000 mejores trigramas filtrados
[('4/', 'beach-head', '2000'), ('bahay', 'kubo', 'natin'), ('bar/boxing', 'ring/vintage', 'store/food'), ('canh', 'chua', 'tom.i'), ('cheese/jewish', 'sliders/patty', 'melt/kosher'), ('dead-eyed', 'gamblers', 'shuffling'), ('donec', 'obviam', 'r

<div style="color:blue">Como podemos ver, hay signos de puntuación al principio o final de los tokens, por eso filtraría también estos ngramas. Para ello, se podrían eliminar estos símbolos de las opiniones, como se realiza a continuación.</div>

In [21]:
#stopwords = stopwords + ['"',"'", '.', ',', ';', ':','-', '(', ')', '!', '?', '-', '‘', '’', '[', ']']

In [22]:
# Eliminar caracteres especiales de los bigramas
opinions_tokens_cleaned = [token.strip('".,;:-():!?-‘’  =\'') for token in opinions_tokens]
                                 
bigram_candidates = BigramCollocationFinder.from_words(opinions_tokens_cleaned)
best_1000_bigram_candidates = bigram_candidates.nbest(bigram_measures.pmi, 1000)

best_1000_bigram_candidates_filtered = filter_candidates(best_1000_bigram_candidates, stopwords)
print("1000 mejores bigramas filtrados")
print(list(best_1000_bigram_candidates_filtered)[1:30])

1000 mejores bigramas filtrados
[('08', 'cakebread'), ('1/16', 'teaspoon'), ('1202', 'mohave'), ('1980s', 'pseudo-asian'), ('1min', '45sec'), ('2-4-1', 'woot'), ('4/', 'beach-head'), ('49er', 'flapjacks'), ('4th-tier', 'mid-90'), ('50¢', 'addtl'), ('65+', 'eyefucking'), ('abe', 'frohman'), ('administrative', 'assistant'), ('advent', 'epicurean'), ('afterglow', 'punctuated'), ('alain', 'keller'), ('alessandro', 'marchesan'), ('altering', 'drugs'), ('ambience=four', 'estrellas'), ('angello', 'dorato'), ('anita', 'bryant'), ('anxiously', 'awaiting'), ('authoritarian', 'overlords'), ('auto', 'auction'), ('avid', 'youth'), ('azucena', 'tovar'), ('bacon/bleu', 'cheese/walnut'), ('bahay', 'kubo'), ('band/art', 'brut/we')]


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong>  Detectar ngramas que cumplen el patrón sintáctico de un sintagma nominal (e.g: adjetivo + nombre en singular/plural y nombre + nombre) (0.5 puntos)
</div>

<div style="color:blue">En este caso también se emplearan algunas de las funiones definidas en la <b>PLA1</b> de la asignatura, para detectar ngramas que cumplen un patrón sintáctico dado.</div>

In [23]:
def patterns_and_parser():
    #Definición de los patterns sintácticos de un nodo N.
    jj_nns_patterns = """
              N: {<JJ> <NN> | <JJ> <NNS>}
              """
    #Definición de los patterns sintácticos de un nodo CN. 
    cn_patterns = """
              CN: {<NN> <NN>}
              """
    #Definición del parser que creará los nodos N del árbol
    jj_nns_parser = nltk.RegexpParser(jj_nns_patterns)
    
    #Definición del parser que creará los nodos CN del árbol    
    cn_parser = nltk.RegexpParser(cn_patterns)
    return jj_nns_patterns, cn_patterns, jj_nns_parser, cn_parser


def get_string_form(tuple_list):
    words = [cti[0] for cti in tuple_list]
    string_form = "_".join(words)
    return string_form


def get_n_and_cn(tagged_tokens):
    #Definición de patterns sintácticos y parser
    n_patterns, cn_patterns, n_parser, cn_parser = patterns_and_parser()
    
    #Creación de los árboles con los nodos N
    n_tree = n_parser.parse(tagged_tokens)
    
    #Creación de los árboles con los nodos CN
    cn_tree = cn_parser.parse(tagged_tokens)
    
    #Listado de las hojas de los nodos N
    n_leaves = [s.leaves() for s in n_tree.subtrees() if s.label() == 'N']
    
    #Listado de las hojas de los nodos CN
    cn_leaves = [s.leaves() for s in cn_tree.subtrees() if s.label() == 'CN'] 

    #Unión de los dos listados
    n_cn_tuples = n_leaves + cn_leaves   
    
    #Conversión de las tuplas que representan las hojas al término
    n_and_cn = [get_string_form(c) for c in n_cn_tuples]
    
    return n_and_cn

In [24]:
#############################################
# SOLUCIÓN                                  #
#############################################
n_and_cn = get_n_and_cn(opinions_tagged_tokens)

In [25]:
print(n_and_cn[0:100])

['absolute_pleasure', 'saturday_morning', 'pretty_sure', 'white_truffle', 'vegetable_skillet', 'bad_reviews', 'own_fault', 'many_people', 'past_sunday', 'sunday_evening', 'baked_spaghetti', 'sweetish_sauce', 'bad_reviewers', 'bad_reviewers', 'serious_issues', 'full_windows', 'earthy_wooden', 'tuesday_evening', 'couple_days', 'fresh_veggies', 'much_i', 'raw_radishes', 'long_time', 'warm_foccacia', 'pomegranate_slices', 'vegetarian_entrees', 'pear_cake', 'next_day', 'green_building', 'grand_opening', 'yelping_soul', 'new_place', 'la_condesa', 'dogfish_shark', 'delicious_meals', 'much_flavor', 'hubbys_mole', 'mahi_mahi', 'burros-_mmmm', 'salsa_bar', 'strawberry_salsa', 'big_wimp', 'hot_peppers', 'yummy_bonus', 'good_food', 'mexican_folk', 'salsa_bar', 'happy_hour', 'great_atmosphere', 'wait_staff', 'apollo_beach', 'unique_talents', 'vietnamese_sandwich', 'sandwich_choices', 'modest_selection', 'baked_goods', 'atm_card', 'limited_time', 'other_things', 'irish_bars', 'wednesday_night', 'fri

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Detectar colocaciones con un modelo de detección de frases, con el módulo Phraser de Gensim. Entrena el modelo con todas las opiniones (0,5 puntos)
</div>

<i>Primer paso</i>: Convertir las opiniones en una lista de phrases. Las phrases no deben ser stopwords. Tampoco deben empezar ni acabar con una stopword. (0.5 puntos)

In [26]:
# Obtenemos las frases del texto
opinions_string = " ".join(df['text'])
opinion_sentences = opinions_string.split('. ')
opinion_sentences[:10]

['My wife took me here on my birthday for breakfast and it was excellent',
 'The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure',
 'Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning',
 'It looked like the place fills up pretty quickly so the earlier you get here the better',
 'Do yourself a favor and get their Bloody Mary',
 "It was phenomenal and simply the best I've ever had",
 "I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it",
 'It was amazing',
 'While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious',
 'It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete']

In [27]:
#############################################
# SOLUCIÓN                                 #
#############################################
from gensim.models import Phrases

# Convertimos las frases en un stream sin stopwords
opinion_sentences_stream = [word_tokenize(opinion.lower()) for opinion in opinion_sentences]
opinion_sentences_stream_no_stopwords = [candidate for candidate in opinion_sentences_stream if (len(candidate) > 0 and stopword_candidate(candidate) != True and not str(candidate) in stopwords) ]

# Convertimos el stream en Phrases
opinion_phrases = Phrases(opinion_sentences_stream_no_stopwords, min_count=1, threshold=2, delimiter=' ')

In [28]:
document_tokens = word_tokenize(opinions_string.lower())
opinion_phrases_no_stopwords = opinion_phrases[document_tokens]

#Imprimimos las primeras 5 Phrases
print(opinion_phrases_no_stopwords[:20])

['my wife', 'took me', 'here', 'on', 'my birthday', 'for breakfast', 'and', 'it was', 'excellent', '.', 'the weather', 'was perfect', 'which made', 'sitting outside', 'overlooking', 'their', 'grounds', 'an absolute', 'pleasure', '.']


## 1.2 Vectorizar palabras y términos (2 puntos)

Exploraremos la vectorización de palabras y términos con el método Word2Vec.

Recordemos que el paquete gensim implementa un método para entrenar modelos Word2Vec.

In [31]:
# Quitar espacios del texto
opinion_phrases_stripped_no_stopwords = [c.strip() for c in opinion_phrases_no_stopwords]
opinion_phrases_stripped_no_stopwords[:10]

['which made',
 'pleasure',
 'like',
 'place',
 'fills up',
 'earlier',
 'better',
 'phenomenal',
 'pretty sure',
 'fresh']

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
Ejercicio:</strong> Obtener targets de las opiniones y sus aspectos utilizando el modelo word2vec (2 puntos)
</div>

<i>Primer paso</i>: Convertir las phrases de cada oración en un token. Lo haremos concatenando los tokens de la phrase con el caracter '_' (e.g: 'scrambled eggs' -> 'scrambled_eggs'). Entonces, en cada oración sustituimos los bigramas que son phrases por la forma tokenizada (e.g: I made scrambled eggs -> I made scrambled_eggs). De esta forma, las colocaciones formarán parte del vocabulario del modelo word2vec que generaremos.

In [33]:
from nltk.util import ngrams
import gensim

# Filter collocations
collocation_phrases = [phrase for phrase in list(set(opinion_phrases_stripped_no_stopwords)) if ' ' in phrase]

def transform_sentence(sentence):
    transformed_sentence = sentence
    n_grams = list(ngrams(nltk.word_tokenize(sentence), 2))
    ngrams_t = [' '.join(gram) for gram in n_grams]
    for ngram in ngrams_t:
        if ngram in collocation_phrases:
            opt = ngram.replace(' ', '_')
            transformed_sentence = transformed_sentence.replace(ngram,opt)
    return transformed_sentence

opinion_sentences_transformed = [transform_sentence(opinion_sentence) for opinion_sentence in opinion_sentences]

opinion_sentences_transformed[:10]

['My wife took me here on my birthday for breakfast and it was excellent',
 'The weather was perfect which_made sitting outside overlooking their grounds an absolute pleasure',
 'Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning',
 'It looked_like the place fills_up pretty quickly so the earlier you get here the better',
 'Do yourself a favor and get their Bloody Mary',
 "It was_phenomenal and simply the best I've ever had",
 "I'm pretty_sure they only use ingredients from their garden and blend them fresh when_you order it",
 'It was_amazing',
 'While EVERYTHING on the menu looks excellent, I had the white_truffle scrambled eggs vegetable skillet and it was tasty and delicious',
 'It came_with 2 pieces_of their griddled bread with was_amazing and it absolutely made the meal complete']

<i>Segundo paso</i>: crear una sentence stream donde todos los tokens de las oraciones están lematizados. Los tokens no pueden ser stopwords ni tener un stopword al inicio o al final. Para simplificar la tarea, consideramos que el lema de una colocación no cambia y su PoS es 'col'. (e.g: ['The guests like scrambled eggs', 'The rooms were dirty'] -> [['the', 'guest', 'like', 'scrambled_eggs], ['the', 'room', 'be ', 'dirty']]). (1 punto)

In [34]:
#############################################
# SOLUCIÓN                                  #
#############################################
from nltk.stem.wordnet import WordNetLemmatizer

postag = {}

def get_wn_pos(pos):
    if re.match(r'^N',pos):
        wn_pos = 'n'
    elif re.match(r'^V',pos):
        wn_pos = 'v'
    else:
        wn_pos = 'n' #En inglés, los lemas de términos que no son verbos ni nombres se obtienen como si fueran
                        #nombres
    return wn_pos

def wnlemmatize(t,postag):
    lemma = ""
    #Definición del lematizador
    lem = WordNetLemmatizer()
    #Si el candidato es monopalabra, se obtiene el lema con el lematizador de WordNet según su PoS
    if ' ' not in t:
        lemma = lem.lemmatize(t,get_wn_pos(postag[0][1]))
    #Si el candidato es multipalabra, obtenemos su lema como si fuera un nombre, aplicando el lematizador de WordNet
    else:
        lemma = lem.lemmatize(t,'n')
    return lemma


def transform_sentence(sentence):
    #Obtenemos los phrases según el modelo de detección de phrases que ha aprendido
    sentence_phrases = opinion_phrases[word_tokenize(sentence.lower())]
    #Quitamos los signos de puntuación de los phrases
    phrases_stripped = [st.strip('".,;:-():!?-‘’ ') for st in sentence_phrases if re.match("^[a-z]+.*", st)]
    #Quitamos stopwords
    phrases_stripped_no_stopwords = [candidate for candidate in phrases_stripped if (len(candidate) > 0 and stopword_candidate(candidate) != True and not str(candidate) in stopwords) ]

    #Hacemos etiquetaje de PoS de los phrases y lo guardamos en un diccionario (postag) 
    for ps in phrases_stripped_no_stopwords:
        postag[ps] = nltk.pos_tag(word_tokenize(ps))
    #Lematizamos los phrases con el lematizador de Wordnet
    phrases_lemmatized = [wnlemmatize(ps, postag[ps]) for ps in phrases_stripped_no_stopwords]
    #Unificamos los phrases
    #phrases_lemmatized_and_unified = [unify(pl) for pl in phrases_lemmatized ]
    return phrases_lemmatized

transformed_sentences = [transform_sentence(ss) for ss in opinion_sentences_transformed ]

print(transformed_sentences[:5])

[['my wife', 'took me', 'my birthday', 'for breakfast', 'it was', 'excellent'], ['the weather', 'was perfect', 'sitting outside', 'overlook', 'ground', 'an absolute'], ['our waitress', 'was excellent', 'our food', 'arrived quickly', 'semi-busy', 'saturday morning'], ['pretty quickly', 'you get'], ['do yourself', 'a favor', 'get', 'their bloody', 'mary']]


<i>Tercer paso</i>: Crear un modelo word2vec de las opiniones lematizadas. El modelo debe llamarse w2v_opinions (0.5 puntos)

In [35]:
#############################################
# SOLUCIÓN                                   #
#############################################
# Creamos el modelo
w2v_opinions = gensim.models.Word2Vec(
        transformed_sentences,
        vector_size=150,
        window=10, 
        min_count= 3,
        workers= 1,
        seed=1 # 
)
# Entrenamos el modelo
w2v_opinions.train(transformed_sentences, total_examples=len(transformed_sentences), epochs=10)

(2441528, 2767150)

<i>Cuarto paso</i>: A partir del vocabulario del modelo word2vec, selecciona posibles aspectos de la opinión (e.g: food) y lista términos semánticamente relacionados con estos aspectos según este modelo. (0.5 puntos)

Obtener el vocabulario:

In [36]:
vocabulary = list(w2v_opinions.wv.key_to_index.keys())

In [37]:
#############################################
# SOLUCIÓN                                   #
#############################################

no_pos_in = ['DT', 'IN', 'PRP', 'CC', 'CD','MD', 'VBG', 'VBD', 'RP', 'RB']

def good_candidate(t,postag):
    v = False
    #Si es multipalabra
    if ' ' in t:
        tl = t.split(' ') #Generamos una lista de tokens
        #el token inicial y el token final deben ser alfabéticos y no pueden estar en la lista de stopwords..
        if re.match("^[a-z]+.*", tl[0]) and re.match("^[a-z]+.*", tl[-1]) and \
           tl[0] not in stopwords and tl[1] not in stopwords:
            #... ni su PoS puede estar en la lista no_pos_in
            if postag[0][1] not in no_pos_in and postag[-1][1] not in no_pos_in:
                v = True
    #Si es monopalabra
    else:
        #debe ser alfabético, y no estar en la lista de stopwords
        if t not in stopwords and re.match("^[a-z]+.*", t):
            #y su PoS no puede estar en la lista no_pos_in
            if postag[0][1] not in no_pos_in:
                v = True
    return v

def phrase_is_term(phrase):
    test = False
    if phrase not in postag:
        pos = nltk.pos_tag(word_tokenize(phrase))
    else:
        pos = postag[phrase]
    if good_candidate(phrase,pos):
        test = True
    return test

terms_vocabulary = [word for word in vocabulary if phrase_is_term(word) == True]
print(terms_vocabulary[1:100])

['food', 'get', 'great', 'time', 'order', 'restaurant', 'make', 'try', 'pizza', 'say', 'go', 'delicious', 'salad', 'u', 'meal', 'drink', 'fry', 'table', 'night', 'sauce', 'sandwich', 'service', 'taste', 'take', 'dish', 'tasty', 'right', 'meat', 'bread', 'thing', 'way', 'dinner', 'menu', 'day', 'amaze', 'area', 'sushi', 'enjoy', 'wait', 'hot', 'visit', 'serve', 'sweet', 'side', 'eat', 'appetizer', 'seat', 'breakfast', 'special', 'taco', 'soup', 'dessert', 'awesome', 'find', 'see', 'something', 'server', 'friend', 'spicy', 'small', 'excellent', 'perfect', 'spot', 'much', 'overall', 'bad', 'happy', 'think', 'steak', 'yummy', 'quality', 'ok', 'want', 'start', 'shrimp', 'recommend', 'best', 'salsa', 'onion', 'place', 'add', 'different', 'many', 'year', 'yes', 'disappointed', 'open', 'item', 'sure', 'guy', 'option', 'oh', 'left', 'anything', 'offer', 'great food', 'tomato', 'price', 'busy']


In [38]:
def get_related_terms(w2v_model, term):
    w2v_tuples = []
    feature_names = terms_vocabulary

    for i in range(0, len(feature_names)):
        if feature_names[i] != term and w2v_model.wv.similarity(term, feature_names[i]) > 0:
            w2v_tuples.append((feature_names[i], w2v_model.wv.similarity(term, feature_names[i])))

    #Se ordenan las tuplas
    w2v_sorted_tuples = sorted(w2v_tuples, key=lambda tup: tup[1], reverse=True)
    labels = ['Term', 'Distance']

    #Se crea dataframe a partir del cual se construirá la tabla
    df = pd.DataFrame.from_records(w2v_sorted_tuples, columns=labels)
    return df

df_related_terms_menu =  get_related_terms(w2v_opinions, "menu")
df_related_terms_menu.head(n=10)

,Term,Distance
0,beer,0.807356
1,list,0.805211
2,special,0.786069
3,offer,0.773176
4,available,0.759095
5,option,0.730790
6,several,0.704829
7,tap,0.700200
8,plate,0.684591
9,drink menu,0.683819


In [39]:
#############################################
# SOLUCIÓN                                   #
#############################################

#Imprimimos ejemplos de aspectos cercanos semánticamente al target según el modelo Word2Vec que el alumno puede 
#encontrar en el dataframe anterior. Entre estos ejemplos está, por ejemplo, 'atmosphere' que tiene un valor de
#similitud semántica alto. El ejercicio consiste en que el alumno elija los ejemplos que considere pertinentes.

df_related_terms_menu =  get_related_terms(w2v_opinions, "food")
df_related_terms_menu.head(n=10)

,Term,Distance
0,inflate,0.690379
1,price,0.683504
2,great food,0.680267
3,great prices,0.636369
4,cheap prices,0.621569
5,ambiance,0.615045
6,good eats,0.599672
7,overall,0.580139
8,food quality,0.580114
9,pho restaurant,0.579262


In [40]:
df_related_terms_menu =  get_related_terms(w2v_opinions, "service")
df_related_terms_menu.head(n=10)

,Term,Distance
0,staff,0.858470
1,ambiance,0.830096
2,great food,0.781090
3,scottsdalish,0.757340
4,ambience,0.753747
5,terrible service,0.753170
6,courteous,0.739609
7,orient,0.736084
8,soothe,0.730038
9,food quality,0.727395


In [41]:
df_related_terms_menu =  get_related_terms(w2v_opinions, "restaurant")
df_related_terms_menu.head(n=10)

,Term,Distance
0,business,0.831953
1,area,0.807573
2,shop,0.787037
3,mall,0.780538
4,community,0.778575
5,scottsdale,0.777343
6,tempe,0.767680
7,din,0.756355
8,arena,0.744803
9,shopping plaza,0.743308


In [42]:
#help(gensim.models.Word2Vec)

<div style="color:blue">Como de puede observar, hemos imprimido la lista de aspectos relacionados para los términos food, restaurant y service.</div>

# 2. Detección de temas (4 puntos)

En estos apartados exploraremos cúales son los tópicos tratados en las opiniones.

## 2.1 Exploración de los temas con WordNet (2 puntos)

En este apartado accederemos a Wordnet a través de la librería nltk.

In [44]:
from nltk.corpus import wordnet as wn

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Comprueba si, según Wordnet, existen aspectos que están alejados semánticamente del sentido del target, aunque en el modelo word2vec sean similares. Compruébalo calculando la similitud de Wu and Palmer entre el sentido de wordnet 'restaurant.n.01' y algunos de sus aspectos. (1 punto)
</div>

In [45]:
#############################################
# SOLUCIÓN                                   #
#############################################
# definimos el sysnet de los términos a calcular
restaurant = wn.synset('restaurant.n.01')
menu = wn.synset('menu.n.01')
food = wn.synset('food.n.01')
quality = wn.synset('quality.n.01')
service = wn.synset('service.n.01')
business = wn.synset('business.n.01')

print("LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'MENU' ES ", restaurant.wup_similarity(menu)) 
print("LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'FOOD' ES ", restaurant.wup_similarity(food)) 
print("LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'QUALITY' ES ", restaurant.wup_similarity(quality)) 
print("LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'SERVICE' ES ", restaurant.wup_similarity(service)) 
print("LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'BUSSINES' ES ", restaurant.wup_similarity(business)) 

LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'MENU' ES  0.11764705882352941
LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'FOOD' ES  0.3076923076923077
LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'QUALITY' ES  0.16666666666666666
LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'SERVICE' ES  0.125
LA DISTANCIA SEMÁNTICA ENTRE 'RESTAURANT' Y 'BUSSINES' ES  0.13333333333333333


<div style="color:blue">Según los resultados, los términos más cercanos son <b>food</b> y <b>restaurant</b>.</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Lista los términos monopalabra del vocabulario de word2vec que no están en Wordnet. Los términos deben ser nombres o adjetivos. ¿Creeis que estos términos son importantes para poder realizar un buen análisis de sentimientos? (1 punto)

In [46]:
#############################################
# SOLUCIÓN                                  #
#############################################
no_wordnet_terms = []

# Verificar si cada término del vocabulario está en WordNet
for term in vocabulary:
    #print(len(term.split(r'_')))
    if len(term.split(r'_')) == 1 and len(term.split(r' ')) == 1 and not wn.synsets(term):
        no_wordnet_terms.append(term)

# Imprimir los términos que no están en WordNet
print("Términos monopalabra del vocabulario de Word2Vec que no están en WordNet:")
print(no_wordnet_terms)

Términos monopalabra del vocabulario de Word2Vec que no están en WordNet:
["n't", 'since', 'would', 'something', 'without', 'anything', 'scottsdale', 'could', 'pho', 'others', 'yum', 'although', 'crunchy', 'tempe', 'anyone', 'omg', 'bruschetta', 'unless', 'delish', 'gelato', 'st', 'apps', 'app', 'take-out', 'asada', 'ya', 'torta', 'hey', 'mojito', 'chris', 'yuck', 'meh', 'yelpers', 'to-go', 'feta', 'queso', 'asu', 'fyi', 'im', 'and/or', 'bianco', 'masala', 'mmmm', 'cibo', 'def', 'dr', 'um', 'alfredo', 'gallo', 'delux', 'thats', 'nigiri', 'everybody', 'definately', 'postino', 'ave', 'boba', 'ipa', 'p.s', 'kimchi', 'in-n-out', 'starbucks', 'yep', 'haha', 'lgo', 'thru', 'sub-par', 'undercooked', 'legit', 'smokey', 'fajitas', 'sakana', 'sooo', 'ahi', 'pollo', 'sandwhich', 'soooo', 'carnitas', 'quinoa', 'mmm', 'taquitos', 'burrata', 'staffperson', 'horchata', 'chimichanga', 'sans', 'fajita', 'resturant', 'francis', 'til', 'didnt', 'dang', 'via', 'gyoza', 'onto', 'hana', 'steve', 'dont', 'am

<div style="color:blue">Sí, porque hay términos que pueden ser muy espécificos del ámbito culinario o expresiones que estén de moda, y que no aparezcan en la lista. Además vemos términos que son, como se comentaba en la teoría de la asignatura (2.3.1 de Extracción de sentimientos y opiniones, Joaqim Moré), acrónimos que se emplean como sinónimos de otras palabras. Por ejemplo la palabra <b>fest, mgr, fam, approx, etc.</b> Algunos otros términos que pueden ser significativos de la lista de arriba son <b>orgasmic, michelin, by-the-glass, etc. </b></div>

## 2.2 LDA (2 puntos)

Recordad que en el notebook del módulo 1 hemos visto la aplicación del método LDA para extraer temas de documentos.

<i>Primer paso</i>: Convertir las opiniones transformadas (opinion_sentences_transformed) en listas de nombres y colocaciones. Esto es necesario ya que los nombres y las colocaciones expresan los temas de las opiniones (e.g: [['wife have breakfast sitting_outside'], [' 'waitress', 'served', 'ceviche']...] -> [['wife', 'breakfast', 'sitting_outside'], ['waitress', 'ceviche']]

In [47]:
def get_noun_and_collocation(sentence):
    nouns_and_collocations = []
    noun_tags = ['NN', 'NNS']
    tokens_pos_tagged = pos_tag(word_tokenize(sentence))
    for tpos in tokens_pos_tagged:
        if '_' in tpos[0]:
            nouns_and_collocations.append(tpos[0])
        elif tpos[1] in noun_tags:
            nouns_and_collocations.append(tpos[0])
    return nouns_and_collocations
            
noun_and_collocation_stream = [get_noun_and_collocation(opinion) for opinion in opinion_sentences_transformed]

noun_and_collocation_stream[:5]

[['wife', 'birthday', 'breakfast'],
 ['weather', 'which_made', 'grounds', 'pleasure'],
 ['waitress', 'food', 'morning'],
 ['looked_like', 'place', 'fills_up'],
 ['favor']]

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Extrae temas a partir de las listas de nombres y colocaciones de cada oración transformada. Experimenta con el parámetro num_topics hasta encontrar un conjunto de temas informativos, asignando nombres a los temas encontrados. (2 puntos)
</div>

In [9]:
#############################################
# SOLUCIÓN                                 #
#############################################
import gensim.corpora as corpora
def lda(terms, num_topics):
    dictionary = corpora.Dictionary(terms)
    #print(dictionary)
    # Creación del corpus
    texts = terms
    # Frecuencia de los términos en cada documento. El formato está en forma de tupla,
    #(índice del término en el diccionario/vocabulario, frecuencia). Por ejemplo, [(0, 2), (1, 1), (2, 1), (3, 1)])
    corpus = [dictionary.doc2bow(text) for text in texts]
    #Creación del modelo.
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, # Frecuencia de los términos en cada documento
                                               num_topics=num_topics, #Número de temas
                                               random_state=1, #Valor de inicio predefinido para conservar 
                                                               #coherencia
                                               id2word = dictionary, #El vocabulario
                                               passes=500) # Cuantos más pases, más consistente el modelo

    return ldamodel

ldamodel_7 = lda(noun_and_collocation_stream, 7)
ldamodel_7.save("lda_7.model")
#ldamodel_7 = gensim.models.ldamodel.LdaModel.load("lda_7.model")

In [52]:
import numpy as np

K = ldamodel_7.num_topics
topicWordProbMat = ldamodel_7.print_topics(num_topics=K, num_words=10)

#La matriz se muestra como un dataframe
columns = ['1','2','3', '4', '5','6','7']
df_topic = pd.DataFrame(columns = columns)
pd.set_option('display.width', 2000)

zz = np.zeros(shape=(1000,K))

last_number=0
DC={}

for x in range (10):
    data = pd.DataFrame({columns[0]:"",
                     columns[1]:"",
                     columns[2]:"",
                     columns[3]:"",
                     columns[4]:"",                                                                               
                     columns[5]:"",                                                                               
                     columns[6]:"",
                            },index=[0])
    df_topic= pd.concat([df_topic, data], ignore_index=True)


for line in topicWordProbMat:
    topic, word = line
    probabilities=word.split("+")
    print(probabilities)
    y=0
    for pr in probabilities:   
        a=pr.split("*")
        df_topic.iloc[y,topic] = a[1]
        if a[1] in DC:
                zz[DC[a[1]]][topic]=a[0]
        else:
            zz[last_number][topic]=a[0]
            DC[a[1]]=last_number
            last_number=last_number+1
            y=y+1

#Ordenar strings con números          
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    def atoi(text):
        return int(text) if text.isdigit() else text

    return [atoi(c) for c in re.split('(\d+)', text)]

df_topic =df_topic.reindex(columns=sorted(df_topic.columns, key=natural_keys))
df_topic

['0.027*"time" ', ' 0.019*"meal" ', ' 0.017*"chicken" ', ' 0.016*"sushi" ', ' 0.013*"sauce" ', ' 0.013*"night" ', ' 0.013*"happy_hour" ', ' 0.012*"tacos" ', ' 0.012*"spicy" ', ' 0.011*"dish"']
['0.044*"menu" ', ' 0.021*"people" ', ' 0.020*"day" ', ' 0.013*"everything" ', ' 0.011*"items" ', ' 0.011*"patio" ', ' 0.011*"quality" ', ' 0.010*"family" ', ' 0.010*"waiter" ', ' 0.009*"night"']
['0.024*"staff" ', ' 0.022*"pizza" ', ' 0.017*"breakfast" ', ' 0.012*"friends" ', ' 0.012*"tables" ', ' 0.011*"decor" ', ' 0.011*"salad" ', ' 0.011*"visit" ', ' 0.010*"something" ', ' 0.008*"one"']
['0.123*"place" ', ' 0.044*"food" ', ' 0.043*"service" ', ' 0.015*"experience" ', ' 0.013*"restaurant" ', ' 0.011*"atmosphere" ', ' 0.009*"we_were" ', ' 0.009*"way" ', ' 0.008*"places" ', ' 0.008*"night"']
['0.035*".." ', ' 0.018*"bread" ', ' 0.015*"sandwich" ', ' 0.013*"years" ', ' 0.013*"side" ', ' 0.010*"pizza" ', ' 0.010*"chips" ', ' 0.009*"things" ', ' 0.009*"salad" ', ' 0.009*"chicken"']
['0.092*"food" '

,1,2,3,4,5,6,7
0,"""time""","""menu""","""staff""","""place""","""..""","""order""","""i"""
1,"""meal""","""people""","""pizza""","""food""","""bread""","""bar""","""meat"""
2,"""chicken""","""day""","""breakfast""","""service""","""sandwich""","""times""","""burger"""
3,"""sushi""","""everything""","""friends""","""experience""","""years""","""drinks""","""fries"""
4,"""sauce""","""items""","""tables""","""restaurant""","""side""","""table""","""stars"""
5,"""night""","""patio""","""decor""","""atmosphere""","""chips""","""minutes""","""bit"""
6,"""happy_hour""","""quality""","""salad""","""we_were""","""things""","""server""","""taste"""
7,"""tacos""","""family""","""visit""","""way""","""chicken""",,"""dishes"""
8,"""spicy""","""waiter""","""something""","""places""",,,"""restaurants"""
9,"""dish""","""night""","""one""","""night""",,,"""shrimp"""


In [50]:
print("Topic with num_topics = 7")
ldamodel_7.show_topic(1, topn=10)

Topic with num_topics = 7


[('menu', 0.04371323),
 ('people', 0.02121672),
 ('day', 0.020214884),
 ('everything', 0.013295376),
 ('items', 0.011493003),
 ('patio', 0.011377151),
 ('quality', 0.011224762),
 ('family', 0.010043187),
 ('waiter', 0.00957674),
 ('night', 0.009346394)]

In [ ]:
ldamodel_4 = lda(noun_and_collocation_stream, 4)
ldamodel_4.save("lda_4.model")
#ldamodel_4 = gensim.models.ldamodel.LdaModel.load("lda_4.model")

In [54]:
K = ldamodel_4.num_topics
topicWordProbMat = ldamodel_4.print_topics(num_topics=K, num_words=10)

#La matriz se muestra como un dataframe
columns = ['1','2','3', '4']
df_topic = pd.DataFrame(columns = columns)
pd.set_option('display.width', 2000)

zz = np.zeros(shape=(1000,K))

last_number=0
DC={}

for x in range (10):
    data = pd.DataFrame({columns[0]:"",
                     columns[1]:"",
                     columns[2]:"",
                     columns[3]:""
                            },index=[0])
    df_topic= pd.concat([df_topic, data], ignore_index=True)


for line in topicWordProbMat:
    topic, word = line
    probabilities=word.split("+")
    print(probabilities)
    y=0
    for pr in probabilities:   
        a=pr.split("*")
        df_topic.iloc[y,topic] = a[1]
        if a[1] in DC:
                zz[DC[a[1]]][topic]=a[0]
        else:
            zz[last_number][topic]=a[0]
            DC[a[1]]=last_number
            last_number=last_number+1
            y=y+1

#Ordenar strings con números          
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    def atoi(text):
        return int(text) if text.isdigit() else text

    return [atoi(c) for c in re.split('(\d+)', text)]

df_topic = df_topic.reindex(columns=sorted(df_topic.columns, key=natural_keys))
df_topic

['0.024*"time" ', ' 0.016*"salad" ', ' 0.015*"chicken" ', ' 0.012*"order" ', ' 0.011*"sauce" ', ' 0.011*"side" ', ' 0.010*"i" ', ' 0.008*"sandwich" ', ' 0.007*"meat" ', ' 0.007*"tacos"']
['0.025*"menu" ', ' 0.011*"bread" ', ' 0.009*"cheese" ', ' 0.009*"dishes" ', ' 0.008*"happy_hour" ', ' 0.008*"everything" ', ' 0.008*"burger" ', ' 0.007*"drink" ', ' 0.007*"wine" ', ' 0.007*"beer"']
['0.021*"place" ', ' 0.020*".." ', ' 0.017*"pizza" ', ' 0.017*"bar" ', ' 0.012*"staff" ', ' 0.012*"people" ', ' 0.010*"night" ', ' 0.010*"table" ', ' 0.010*"minutes" ', ' 0.009*"order"']
['0.089*"food" ', ' 0.058*"place" ', ' 0.028*"service" ', ' 0.019*"restaurant" ', ' 0.010*"experience" ', ' 0.009*"sushi" ', ' 0.008*"stars" ', ' 0.008*"things" ', ' 0.008*"bit" ', ' 0.008*"years"']


,1,2,3,4
0,"""time""","""menu""","""place""","""food"""
1,"""salad""","""bread""","""..""","""service"""
2,"""chicken""","""cheese""","""pizza""","""restaurant"""
3,"""order""","""dishes""","""bar""","""experience"""
4,"""sauce""","""happy_hour""","""staff""","""sushi"""
5,"""side""","""everything""","""people""","""stars"""
6,"""i""","""burger""","""night""","""things"""
7,"""sandwich""","""drink""","""table""","""bit"""
8,"""meat""","""wine""","""minutes""","""years"""
9,"""tacos""","""beer""","""order""",


In [55]:
print("Topic with num_topics = 4")
ldamodel_4.show_topic(1, topn=10)

Topic with num_topics = 4


[('menu', 0.025092797),
 ('bread', 0.010584909),
 ('cheese', 0.008714047),
 ('dishes', 0.008699143),
 ('happy_hour', 0.0077446573),
 ('everything', 0.0076329987),
 ('burger', 0.007539459),
 ('drink', 0.007326105),
 ('wine', 0.0072688516),
 ('beer', 0.007145796)]

<div style="color:blue">Viendo los resultados, seleccionaré el modelo con num_topics = 7 </b></div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Ejercicio opcional:</strong> Si consideramos también los verbos, ¿mejoraríamos los resultados? Justifica la respuesta mostrando LDA si consideramos los verbos, además de los nombres y colocaciones.
</div>

In [60]:
#############################################
# SOLUCIÓN                                 #
#############################################
def get_noun_verbs_and_collocation(sentence):
    nouns_and_collocations = []
    noun_tags = ['NN', 'NNS', 'VBD', 'VBG', 'VM' ]
    tokens_pos_tagged = pos_tag(word_tokenize(sentence))
    for tpos in tokens_pos_tagged:
        if '_' in tpos[0]:
            nouns_and_collocations.append(tpos[0])
        elif tpos[1] in noun_tags:
            nouns_and_collocations.append(tpos[0])
    return nouns_and_collocations
            
noun_verb_and_collocation_stream = [get_noun_verbs_and_collocation(opinion) for opinion in opinion_sentences_transformed]

noun_verb_and_collocation_stream[:5]

[['wife', 'took', 'birthday', 'breakfast', 'was'],
 ['weather',
  'was',
  'which_made',
  'sitting',
  'overlooking',
  'grounds',
  'pleasure'],
 ['waitress', 'was', 'food', 'arrived', 'morning'],
 ['looked_like', 'place', 'fills_up'],
 ['favor']]

In [ ]:
ldamodel_new = lda(noun_verb_and_collocation_stream, 7)
ldamodel_new.save("lda_verb.model")
#ldamodel_new = gensim.models.ldamodel.LdaModel.load("lda_verb.model")

In [62]:
print("Nuevo modelo con verbos y num_topics = 7")
ldamodel_new.show_topic(1, topn=10)

Nuevo modelo con verbos y num_topics = 7


[('was', 0.029652366),
 ('times', 0.020628674),
 ('place', 0.019048363),
 ('had', 0.018653965),
 ('go_back', 0.009388573),
 ('price', 0.009331025),
 ('wait', 0.0092755575),
 ('decor', 0.009095241),
 ('would_be', 0.008546606),
 ('first_time', 0.007924653)]

In [8]:
 !pip install pyLDAvis

In [7]:
import pyLDAvis.gensim
import gensim.corpora as corpora

texts = data_prueba_csv
texts = [text for text in texts if len(text) > 0]
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(topic_model=ldamodel_new, corpus=corpus, dictionary=dictionary)
#TypeError: drop() takes from 1 to 2 positional arguments but 3 were given

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.134885  0.056317       1        1  18.724604
0     -0.291262 -0.138211       2        1  15.677214
1     -0.087765 -0.113505       3        1  14.252538
4      0.156386 -0.134779       4        1  13.912512
2     -0.160825  0.188946       5        1  13.377197
5      0.191576 -0.107004       6        1  12.201221
3      0.057004  0.248236       7        1  11.854715, topic_info=               Term          Freq         Total Category  logprob  loglift
12      looked_like   3302.000000   3302.000000  Default  30.0000  30.0000
17           garden   3441.000000   3441.000000  Default  29.0000  29.0000
114         ordered   1539.000000   1539.000000  Default  28.0000  28.0000
3           sitting  10491.000000  10491.000000  Default  27.0000  27.0000
61           server   1967.000000   1967.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
1185     left_super     98.784882    159.327159   Topic7  -5.5839   1.6544
19            order    276.803981   3624.500908   Topic7  -4.5536  -0.4397
184            Drop    115.851901    377.521747   Topic7  -5.4246   0.9511
263   everything_on     93.113962    378.841857   Topic7  -5.6430   0.7291
617             hey     86.819771    258.112019   Topic7  -5.7130   1.0429

[333 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
438       5  0.997689    2_people
184       5  0.691351        Drop
184       7  0.307267        Drop
23        1  0.297089  EVERYTHING
23        3  0.106244  EVERYTHING
...     ...       ...         ...
2393      7  0.990588        word
663       2  0.996485     working
499       5  0.994401    would_be
704       3  0.995587        yelp
401       1  0.993665       yummy

[372 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 2, 5, 3, 6, 4])

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

print("La incorporación de verbos mejora o no los resultados?")

<div style="color:blue">Bajo mi punto de vista, la incorporación de verbos no mejora los resultados, si no que introduce más ruido.</div>

# 3. Clasificación (2 puntos)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Crea un clasificador automático de opiniones positivas y negativas. (0.75 puntos)
</div>

<i>Primer paso</i>: realizamos dos listas. Una con los textos y otra con las etiquetas de valoración (0 y 1)

In [19]:
#############################################
# SOLUCIÓN                                  #
#############################################
opinions = df['text'].to_list()
labels = df['sentiment'].to_list()

<i>Segundo paso</i>: Vectorizamos las opiniones con un vectorizador tf.idf. Usad 'word' como analyzer (0.25 punts)

In [20]:
#############################################
# SOLUCIÓN                                 #
#############################################
from sklearn.feature_extraction.text import TfidfVectorizer

tif_vectorizer = TfidfVectorizer(
    analyzer= 'word')
tif_vectorizer_fit = tif_vectorizer.fit_transform(opinions)
X = tif_vectorizer_fit.toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

<i>Tercer paso</i>: Preparamos el corpus de entrenamiento y evaluación (0.25 puntos)

In [21]:
#############################################
# SOLUCIÓN                                  #
#############################################
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X, labels, train_size=0.80, random_state=999)

<i>Cuarto paso</i>: Entrenar al clasificador con Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
log_model = classifier.fit(X=X_train, y=y_train)

Quinto paso<i>: Utilizar el modelo entrenado para predecir la categoría 1 (positivo) o 0 (negativo) de las opiniones del conjunto de test y mostrar las palabras más informativas para cada categoría. (0.25 puntos)

In [25]:
np.where(classifier.classes_==1, 'Positivo', 'Negativo')

array(['Negativo', 'Positivo'], dtype='<U8')

In [26]:
#############################################
# SOLUCIÓN                                  #
#############################################
y_pred = log_model.predict(X_test)

print("\nCLASES POSITIVAS/NEGATIVAS DEL CORPUS DE TEST:\n===============")

print(y_pred)

def most_informative_feature_for_binary_classification(vectorizer, classifier, n=10):
    class_labels = np.where(classifier.classes_==1, 'Positivo', 'Negativo')
    feature_names = vectorizer.get_feature_names_out()
    topn_class1 = sorted(zip(classifier.coef_[0], feature_names))[:n]
    topn_class2 = sorted(zip(classifier.coef_[0], feature_names))[-n:]

    for coef, feat in topn_class1:
        print (class_labels[0], coef, feat)

    for coef, feat in reversed(topn_class2):
        print (class_labels[1], coef, feat)

print("\nFEATURES MÁS INFORMATIVOS EN LAS OPINIONES POSITIVA Y NEGATIVAS:\n===============")

print(most_informative_feature_for_binary_classification(tif_vectorizer, classifier))



CLASES POSITIVAS/NEGATIVAS DEL CORPUS DE TEST:
[1 1 1 ... 1 1 1]

FEATURES MÁS INFORMATIVOS EN LAS OPINIONES POSITIVA Y NEGATIVAS:
Negativo -4.376775576963335 not
Negativo -3.4084638498683764 no
Negativo -2.8463753188228496 ok
Negativo -2.4902612215150466 bland
Negativo -2.360421604773643 horrible
Negativo -2.347624536673859 bad
Negativo -2.3213252892895766 was
Negativo -2.3093270259990972 wasn
Negativo -2.288174295951619 mediocre
Negativo -2.2322553757462384 better
Positivo 5.33728051728932 great
Positivo 3.3127160358936543 delicious
Positivo 3.142836715113883 good
Positivo 3.040001351761747 love
Positivo 2.7845127848352056 best
Positivo 2.7184735996650833 amazing
Positivo 2.6688238076751256 and
Positivo 2.6687014875332205 excellent
Positivo 2.593323809998095 awesome
Positivo 2.553833822136378 definitely
None


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercicio:</strong> Muestra sobre qué aspectos se hacen valoraciones negativas. (0.75 puntos)
</div>

<i>Primer paso<i>: Elige dos palabras más informativas de la categoría 0 y toma un conjunto de opiniones en las que aparezcan estas palabras. Preprocesa las opiniones quitando los caracteres de salto de línea (0.25 puntos)

In [27]:
opinions_preprocessed = [opinion.replace(r"(\\|\n|\\')", ' ') for opinion in opinions]
opinions_preprocessed[0:5]
worst_words = ["horrible", "mediocre"]

opinions_filtered = [opinion for opinion in opinions_preprocessed if any(word in opinion for word in worst_words)]
opinions_filtered[0:5]

["I've eaten here many times, but none as bad as last night. Service was excellent, and highly attentive. Food, absolutely horrible. My expectation was they would serve a steak on par with their seafood. After all, they were charging 39 bucks for a ribeye. What I was hoping for was a 1- 1-1/2' thick steak, cooked Pittsburgh style as I had ordered. What I got a a 3/4 in thick piece of meat that was mostly fat, gristle, and in no way resembled Pittsburgh Style. Salad, similar to something you could get at Chick Filet Veggies, blah. Bread basket, ample, but day old, and if not, it certainly wasn't fresh. In addition to bad food, we were crammed into a small room where we were nuts to butts with 6 other tables, listening to conversations ranging from someone's recent bout with pinkeye, and another couple who elected to speak entirely in French, until the waiter showed up, then it was like they turned off the French switch and suddenly began speaking English. I've had it with this place. If

<i>Segundo paso<i>: Utiliza el diccionario de opiniones (archivo AFINN-111) para extraer la polaridad de cada opinión como la media de los valores de las opinion words del texto. (0.25 puntos)

In [28]:
#############################################
# SOLUCIÓN                                  #
#############################################
#Creamos una lista de opinion words y pesos a partir del diccionario AFINN
opinion_words_file = 'AFINN-111.txt'

opinion_words = []

with open(opinion_words_file, 'r') as of:
    ol = of.readlines()
    for wo in ol:
        w, o = wo.strip().split('\t')
        opinion_words.append([w,o])

opinion_words_df = pd.DataFrame(opinion_words, columns =['opinion_word', 'polarity'])
head(opinion_words_df)

[['abandon', '-2'],
 ['abandoned', '-2'],
 ['abandons', '-2'],
 ['abducted', '-2'],
 ['abduction', '-2'],
 ['abductions', '-2'],
 ['abhor', '-3'],
 ['abhorred', '-3'],
 ['abhorrent', '-3'],
 ['abhors', '-3']]

In [43]:
def is_np(candidate):
    test = False
    tokens = candidate.split()
    tagged_tokens = nltk.pos_tag(tokens)
    if len(tagged_tokens) > 1:
        PoS_initial = tagged_tokens[0][1][:2]
        PoS_final = tagged_tokens[-1][1][:2]
        if ((PoS_initial == 'NN' or PoS_initial == 'JJ') and (PoS_final == 'NN' or PoS_final == 'NS')):
            test = True
    else:
        if len(candidate) > 1 and tagged_tokens[0][1][:2] == 'NN' or tagged_tokens[0][1][:2] == 'NS':
            test = True 
    return test

def get_np_candidates(text):
    #Definimos las métricas que evaluaran si un n-grama puede ser una collocation
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    #Tokenizamos y obtenemos los tokens del texto
    tokens = [w for w in word_tokenize(text.lower())]
    #Búsqueda de bigramas
    bigramfinder = BigramCollocationFinder.from_words(tokens)
    #Se toman los bigramas que no tienen signos de puntuación, etc.
    bigramfinder.apply_word_filter(lambda w: (re.match(r'\W', w)))
    #N mejores candidatos a ser colocaciones, tras pasar por el cálculo del PMI
    bigram_candidates = bigramfinder.nbest(bigram_measures.pmi,100)
    #Transformación de la tupla del bigrama a collocation
    collocation_candidates = [" ".join(bc) for bc in bigram_candidates]
    #Elegimos los candidatos que son sintagmas nominales
    np_candidates = [c for c in tokens + collocation_candidates if is_np(c) == True]
    return np_candidates

opinion = "I read all the good reviews of this restaurant , and wanted to try it, as it's right up the street from where I live. I don't think I hit a bad night, it was slow, the service was horrible, they confused our order with the table next to us. He put plates of food down and walked off, the order had nothing to do with ours. We tried to get his attention,gave up and took a bite. OMG, it was AWFUL,fried fish that tasted so fishy that I couldn't eat it. At that point, the table next to us realized it was their meal , and got up and walked out . Finally I got the waiters attention, he took the plates ,gave us the correct meal, and didn't offer to do anything about all of our time he had wasted, let alone the mistake. Our real meal was O.K., I tried the tamales, wasn't blown away,my boyfriend said his steak was so-so. I don't think I would come back again."
np_candidates = get_np_candidates(opinion)

In [96]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

polarity_list = []

for i in range (0, len(opinions_filtered)):
    total_polarity = 0
    total_values  = 0
    opinion_words_list = []
    opinion =  opinions_filtered[i]
    #token_list =get_np_candidates(opinion.lower())    
    #print("token_list", token_list)
    for ow in word_tokenize(opinion.lower()):
        if opinion_words_df['opinion_word'].isin([ow]).any():
            #print(ow)
            index = opinion_words_df['opinion_word'][opinion_words_df['opinion_word'] == ow].index[0]
            #print("index", int(opinion_words_df['polarity'][index]))
            total_values += 1
            total_polarity += int(opinion_words_df['polarity'][index])
            opinion_words_list.append(ow)
    #print(total_values)
    #print(total_polarity)
    if(total_values > 0):
        mean_polarity = total_polarity / total_values
    else:
        mean_polarity = 0
    polarity_list.append({"polarity": mean_polarity, "opinion": i, "words":opinion_words_list })           

Tercer paso: Selecciona opiniones con polaridad negativa que ejemplifiquen los aspectos peor valorados. Comenta cuáles son estos aspectos (0.25 puntos)

In [99]:
polarity_list = pd.DataFrame(polarity_list)
print(polarity_list.sort_values(by=['polarity'])[1:10])
opiniones = polarity_list.sort_values(by=['polarity'])[1:10]["opinion"]

     polarity  opinion                                              words
6   -3.000000        6                                         [horrible]
107 -3.000000      107                                         [horrible]
9   -3.000000        9                               [horrible, horrible]
141 -2.333333      141                             [bad, horrible, waste]
97  -2.000000       97                                  [horrible, avoid]
118 -1.800000      118         [horrible, smile, gross, worst, desperate]
3   -1.750000        3  [good, bad, horrible, confused, awful, wasted,...
93  -1.714286       93    [hate, horrible, no, no, pay, no, disappointed]
108 -1.500000      108            [good, wrong, no, disdain, awful, fuck]


In [100]:
opinions_filtered[6]

"Service was horrible and food was mediocre at best~ won't be going back"

In [101]:
opinions_filtered[107]

'This place is over priced and the service is horrible. I went there with my feiends and the server was very rude and degrading!'

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################
opinion_words_df['opinion_word'][opinion_words_df['opinion_word'] == "abhors"].index[0]
opinion_words_df['polarity'][5]

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Obtén los resultados de las métricas de evaluación del clasificador basado en regresión logística. Obtén también los resultados de las métricas de evaluación cuando el modelo del clasificador es diferente. Por ejemplo, un modelo SVM (0.25 puntos)
</div>

In [103]:
#############################################
# SOLUCIÓN MODELO REGRESIÓN LOGÍSTICA              #
#############################################

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#Modelo elegido: logistic regression
logreg_model = LogisticRegression()
#Entrenamiento
logreg_model.fit(X=X_train, y=y_train)
#Clasificación
y_pred = logreg_model.predict(X_test)

#Métricas de evaluación
#https://stats.stackexchange.com/questions/117654/what-does-the-numbers-in-the-classification-report-of-sklearn-mean
bm = metrics.classification_report(y_test, y_pred, labels=[0,1])

print(bm)

              precision    recall  f1-score   support

           0       0.94      0.46      0.62       240
           1       0.88      0.99      0.93       931

    accuracy                           0.88      1171
   macro avg       0.91      0.73      0.77      1171
weighted avg       0.89      0.88      0.87      1171



In [105]:
#############################################
# SOLUCIÓN MODELO SVM                       #
#############################################
import sklearn.svm

classifier = sklearn.svm.LinearSVC()
#Entrenamiento
svm_model = classifier.fit(X=X_train, y=y_train)
#Clasificación
y_pred = svm_model.predict(X_test)

svm = metrics.classification_report(y_test, y_pred, labels=[0,1])

print(svm)


              precision    recall  f1-score   support

           0       0.87      0.70      0.77       240
           1       0.93      0.97      0.95       931

    accuracy                           0.92      1171
   macro avg       0.90      0.84      0.86      1171
weighted avg       0.91      0.92      0.91      1171



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Compara los dos modelos en función de estas métricas de evaluación. (0.25 puntos)
</div>

In [1]:
#############################################
# SOLUCIÓN                                  #
#############################################
print("Decir cuál de los dos modelos tiene mejores valores de precision recall y f1.\nComentar si las diferencias son significativas y apuntar las causas por las cuales un modelo da mejores resultados que el otro.")

Decir cuál de los dos modelos tiene mejores valores de precision recall y f1.
Comentar si las diferencias son significativas y apuntar las causas por las cuales un modelo da mejores resultados que el otro.


<div style="color:blue">Según los resultados que vemos arriba, el modelo que mejores resultados da es SVM </div>

# Bibliografía

- Joaquim Moré, Cómo interpretar y analizar automáticamente la información textual
- PLA1, Scripts de los métodos explicados en el módulo
- Joaquim Moré, Extracción de sentimientos y opiniones
- PLA2, Scripts de los métodos explicados en el módulo
- Joaquim Moré, Evaluación de la calidad de los sistemas de reconocimiento de sentimientos
- PLA3, Scripts de los métodos explicados en el módulo
- https://campus.datacamp.com/courses/introduction-to-natural-language-processing-in-python/regular-expressions-word-tokenization?ex=1

- https://dplyr.tidyverse.org/reference/case_when.html
- https://datatofish.com/numpy-array-to-pandas-dataframe/